In [ ]:
import os
import itertools
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sys
sys.path.append("../")
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Ground_truth_oracles.TF_binding_landscape_models import *

In [ ]:
def is_sequence_a_peak(model, sequence, peak_dict, alphabet="AGTC"):
    if sequence in peak_dict:
        return peak_dict[sequence]
    neighbor=[s for s in sequence]
    sequence_fitness=model.get_fitness(sequence)
    for position in range(len(sequence)):
        for aa in alphabet:
            if aa!=sequence[position]:
               neighbor[position]=aa
               neighbor_string="".join(neighbor)
               if sequence_fitness<model.get_fitness(neighbor_string):
                  peak_dict[sequence] = 0
                  return 0
               elif sequence_fitness>model.get_fitness(neighbor_string):
                  peak_dict[neighbor_string] = 0
               neighbor[position]=sequence[position] # reset 
    peak_dict[sequence] = 1
    return 1

def get_all_peaks(landscape, alphabet='AGTC'):
    peaks=set()
    peak_dict = {}
    for ind, seq in enumerate(itertools.product(alphabet, repeat=14)):
        if ind % 10000 == 0:
            print('Processed {} sequences and found {} peaks'.format(ind, len(peaks)))
            print(len(peak_dict))
        seq = ''.join(seq)
        if is_sequence_a_peak(landscape, seq, peak_dict, alphabet):
           peaks.add(seq)
    return peaks

if not os.path.isdir('../peaks'):
    os.mkdir('../peaks')

In [ ]:
rna_landscape_constructor_1=RNA_landscape_constructor()
rna_landscape_constructor_1.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", 
                                      landscapes_to_test = [0])
landscape1 = next(rna_landscape_constructor_1.generate_from_loaded_landscapes())
rna_landscape_constructor_2=RNA_landscape_constructor()
rna_landscape_constructor_2.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", 
                                      landscapes_to_test = [12])
landscape2 = next(rna_landscape_constructor_2.generate_from_loaded_landscapes())

In [ ]:
peaks_1 = get_all_peaks(landscape1["landscape_oracle"], 'UGTC')
pickle.dump(peaks_1, open('../peaks/peaks_B1L14RNA1.pkl'))

In [ ]:
peaks_2 = get_all_peaks(rna_landscape_constructor_2["landscape_oracle"], 'UGTC')
pickle.dump(peaks_2, open('../peaks/peaks_B2L14RNA1+2.pkl'))